#### Please upvote if you find the notebook useful (and don't forget about github star :) )

# Step 0.0. Install [LightAutoML](https://github.com/sberbank-ai-lab/LightAutoML)

In [ ]:
pip install -U lightautoml

# Step 0.1. Import necessary libraries 

In [ ]:
# Standard python libraries
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.dataset.roles import NumericRole

# Step 0.2. Parameters 

In [ ]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 8 * 3600 # Time in seconds for automl run
TARGET_NAME = 'target'

# Step 0.3. Data load 

In [ ]:
%%time

train_data = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
train_data.head()

In [ ]:
test_data = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
test_data.head()

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
submission.head()

# ========= AutoML preset usage =========


## Step 1. Create Task

In [ ]:
%%time

task = Task('multiclass',)

## Step 2. Setup columns roles

In [ ]:
%%time

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
}

## Step 3. Train on full data 

In [ ]:
%%time 

automl = TabularAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {
                           'use_algos': [['linear_l2', 'lgb_tuned', 'cb_tuned']],
                           'return_all_predictions': True,
                           'weighted_blender_max_nonzero_coef': 0.0
                       },
                       tuning_params = {'max_tuning_time': 1800},
                       reader_params = {'n_jobs': N_THREADS},
                       config_path = '../input/lightautoml-configs/conf_0_sel_type_0.yml'
                       )
oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

## Step 4. Predict for test data

In [ ]:
%%time

test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

# print('Check scores...')
# print('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))

## Step 5. Combine all OOF and test preds into train and test data

In [ ]:
columns = ['PREDS_Linear_' + str(i) for i in range(1, 10)] + \
          ['PREDS_LGBM_' + str(i) for i in range(1, 10)] + \
          ['PREDS_CB_' + str(i) for i in range(1, 10)] + \
          ['PREDS_NN_' + str(i) for i in range(1, 10)]
columns

In [ ]:
oof_nn = pd.read_csv('../input/python-keras-nn-residual/OOF_nn_predict.csv')
test_nn = pd.read_csv('../input/python-keras-nn-residual/nn_predict.csv')

In [ ]:
OOF_preds = pd.DataFrame(np.hstack((oof_pred.data, oof_nn.drop('id', axis = 1))), columns = columns)
TEST_preds = pd.DataFrame(np.hstack((test_pred.data, test_nn.drop('id', axis = 1))), columns = columns)

In [ ]:
for col in columns:
    train_data[col] = OOF_preds[col]
    test_data[col] = TEST_preds[col]

In [ ]:
test_data.head()

## Step 6. Train LightAutoML model on dataset with preds

In [ ]:
%%time 

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
    NumericRole(np.float32, prob = True): columns
}

automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {
                           'use_algos': [['lgb_tuned', 'cb_tuned']],
                       },
                       tuning_params = {'max_tuning_time': 1800},
                       reader_params = {'n_jobs': N_THREADS},
                       configs_list = ['../input/lightautoml-configs/conf_1_sel_type_1.yml'],
                       max_runs_per_config=1
                       )
oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

In [ ]:
# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (20, 10), grid = True)

## Step 7. Predict for test data and check score

In [ ]:
test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

print('Check scores...')
print('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))

## Step 8. Prepare submission

In [ ]:
submission.iloc[:, 1:] = test_pred.data
submission.to_csv('lightautoml_2lvl_8hours_with_nn_oofs.csv', index = False)

In [ ]:
submission